### PREM RAG

In [1]:
!pip install pymupdf faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 860.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [2]:
!pip install openai>=1.0.0

In [3]:
from google.colab import drive
import os
import fitz
import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer
from openai import OpenAI

In [4]:
client = OpenAI(api_key="YOUR-API-KEY")


In [5]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
def estrai_testo_pdf(cartella):
    testi = {}
    for nome_file in os.listdir(cartella):
        if nome_file.endswith(".pdf"):
            path = os.path.join(cartella, nome_file)
            doc = fitz.open(path)
            testo = ""
            for pagina in doc:
                testo += pagina.get_text()
            testi[nome_file] = testo
    return testi

In [7]:
CARTELLA_DISCIPLINARI = '/content/drive/MyDrive/DOC_DOCG'
testi_pdf = estrai_testo_pdf(CARTELLA_DISCIPLINARI)
nomi_file = list(testi_pdf.keys())
corpus = list(testi_pdf.values())
print(f"📝 Trovati {len(corpus)} disciplinari")

📝 Trovati 410 disciplinari


In [8]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embedding = model.encode(corpus, show_progress_bar=True)

index = faiss.IndexFlatL2(embedding[0].shape[0])
index.add(np.array(embedding))
print("✅ Indice FAISS creato.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

✅ Indice FAISS creato.


In [9]:
faiss.write_index(index, "/content/drive/MyDrive/DOC_DOCG/Dati/index_dati.faiss")
with open("/content/drive/MyDrive/DOC_DOCG/Dati/metadata.json", "w") as f:
    json.dump(nomi_file, f)

In [10]:
def query_rag(domanda, top_k=3):
    emb_domanda = model.encode([domanda])
    distanze, indici = index.search(np.array(emb_domanda), top_k)
    contesti = [corpus[i] for i in indici[0]]
    fonti = [nomi_file[i] for i in indici[0]]
    return "\n\n".join(contesti), fonti

In [29]:
domanda_vitigni = "Quali sono i vitigni ammessi nella DOCG Chianti Classico? puoi rispondere solo con un elenco dei vitigni? puoi indicare la percentuale massima e se presente la minima?"
domanda_tipologie = "Quali tipologie sono ammesse nella DOCG barolo? puoi rispondere solo con un elenco delle tipologie?"
contesto, fonti = query_rag(domanda_tipologie, top_k=1)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Sei un esperto di disciplinari del vino italiano."},
        {"role": "user", "content": f"Domanda: {domanda_tipologie}\n\nContesto:\n{contesto}"}
    ]
)

print(response.choices[0].message.content)

- Barolo
- Barolo riserva
- Barolo con menzione geografica aggiuntiva
